In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [2]:
df = pd.read_csv('./new_data/product_model_serial_numbers.csv')
fn_df = pd.read_csv('./new_data/filename_id_lookup.csv', names=['SN', 'FILE_NAME', 'example'])

In [3]:
df.head()

,SERVICE_JOB_ID,FILE_NAME,example,PRODUCT_LINE,MODEL_NUMBER,SERIAL_NUMBER
0,SJ48132474,image-pre-inspection-proof-of-purchase-72fddee...,data/Laundry/SJ48132474/image-pre-inspection-p...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M
1,SJ48132474,image-pre-inspection-proof-of-purchase-bcd8901...,data/Laundry/SJ48132474/image-pre-inspection-p...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M
2,SJ48132474,image-pre-inspection-serial-number-a2dc2e0b-7a...,data/Laundry/SJ48132474/image-pre-inspection-s...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M
3,SJ48132474,image-pre-inspection-serial-number-d843dfb6-fb...,data/Laundry/SJ48132474/image-pre-inspection-s...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M
4,SJ48144276,image-pre-inspection-proof-of-purchase-8e82720...,data/Laundry/SJ48144276/image-pre-inspection-p...,HOME LAUNDRY GAS,GTD42GASJ2WW,GTD42GASJ2WW


In [4]:
fn_df.head()

,SN,FILE_NAME,example
0,1,image-pre-inspection-serial-number-e251c60d-82...,data/SJ48263644/image-pre-inspection-serial-nu...
1,2,image-pre-inspection-serial-number-06b7b5d3-63...,data/SJ49160346/image-pre-inspection-serial-nu...
2,3,image-pre-inspection-serial-number-cdabb4cc-13...,data/SJ48161309/image-pre-inspection-serial-nu...
3,4,image-pre-inspection-proof-of-purchase-03327aa...,data/SJ48161309/image-pre-inspection-proof-of-...
4,5,image-pre-inspection-serial-number-f4accc50-1f...,data/SJ48326900/image-pre-inspection-serial-nu...


In [5]:
df.shape

(3347, 6)

In [6]:
df1 = df.dropna()
print(f"Without duplicate model number - # of dataset = {df1.drop_duplicates(subset='MODEL_NUMBER', keep='last').shape}")
df1.shape

Without duplicate model number - # of dataset = (357, 6)


(2424, 6)

In [7]:
fn_df.shape

(3303, 3)

In [8]:
df1['mod_num_len'] = df1['MODEL_NUMBER'].str.len()
df1.head()

<ipython-input-8-4d7de6371267>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['mod_num_len'] = df1['MODEL_NUMBER'].str.len()


,SERVICE_JOB_ID,FILE_NAME,example,PRODUCT_LINE,MODEL_NUMBER,SERIAL_NUMBER,mod_num_len
0,SJ48132474,image-pre-inspection-proof-of-purchase-72fddee...,data/Laundry/SJ48132474/image-pre-inspection-p...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M,14
1,SJ48132474,image-pre-inspection-proof-of-purchase-bcd8901...,data/Laundry/SJ48132474/image-pre-inspection-p...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M,14
2,SJ48132474,image-pre-inspection-serial-number-a2dc2e0b-7a...,data/Laundry/SJ48132474/image-pre-inspection-s...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M,14
3,SJ48132474,image-pre-inspection-serial-number-d843dfb6-fb...,data/Laundry/SJ48132474/image-pre-inspection-s...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M,14
4,SJ48144276,image-pre-inspection-proof-of-purchase-8e82720...,data/Laundry/SJ48144276/image-pre-inspection-p...,HOME LAUNDRY GAS,GTD42GASJ2WW,GTD42GASJ2WW,12


In [9]:
# total number of unique devices 
len(df1['MODEL_NUMBER'].unique())

357

In [18]:
import re
r = re.compile(r'[D|R|W][A-Z|0-9]*[0-9]*\/*[A-Z|0-9]*')
other_mod_num = df1[df1.MODEL_NUMBER.apply(lambda x: bool(not r.match(x)))]['MODEL_NUMBER'].unique()
other_mod_num

array(['GTD42GASJ2WW', 'GTX22GASK0WW', 'GTD33GASK0WW', '00000000',
       'EFMG527UIW1', 'GFD65GSPN0SN', 'GTX42GASJ2WW', 'GFD85GSSN0WW',
       'GTW680BSJ0WS', 'EFMG627UTT1', 'SHPM65W55N/11', 'GTWN3000M',
       'GUD27GSSMWW', 'GTD42GASJWWW', 'KDFE104HBL1', 'GFP1528SN0WW',
       'NED4655EW1', '1: EFLS527UIW', '796.31512211', 'HTX24GASKOWS',
       'HTX24GASK0WS', 'FGID2466QF7A', 'GTD42GWW', 'MHWZ600TW01',
       'LDT5678ST.DUA413TT', 'GFD65GSSN0WW', 'EFMG627UIW1', 'FFLE4033QT0',
       'GTW460ASJ9WW', 'GTD33GASK', 'GFD65GSPNSN', 'GTD45GASJ2WS',
       'JCD2389DEW', 'FRS26KF6EM1', 'FFU20F2VW0', 'KFCP22EXMP1',
       'LGHK2336TF0', 'LMXS30776S-04', 'GSC25C4EYY01', '106-57943700',
       'MFI2568AES'], dtype=object)

In [38]:
other_mod_num_files = df1[df1['MODEL_NUMBER'].isin(other_mod_num)].drop_duplicates('MODEL_NUMBER')['example']
fn_df[fn_df['example'].isin(other_mod_num_files)].to_csv("non-samsung.csv", index=False)

In [39]:
# length of model number range
df1['mod_num_len'].unique()

array([14, 12,  8, 11, 10, 13,  9,  6, 16, 18])

In [40]:
df1['sn_len'] = df1['SERIAL_NUMBER'].str.len()
df1.head()

<ipython-input-40-931adde57f6b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['sn_len'] = df1['SERIAL_NUMBER'].str.len()


,SERVICE_JOB_ID,FILE_NAME,example,PRODUCT_LINE,MODEL_NUMBER,SERIAL_NUMBER,mod_num_len,sn_len
0,SJ48132474,image-pre-inspection-proof-of-purchase-72fddee...,data/Laundry/SJ48132474/image-pre-inspection-p...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M,14,15
1,SJ48132474,image-pre-inspection-proof-of-purchase-bcd8901...,data/Laundry/SJ48132474/image-pre-inspection-p...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M,14,15
2,SJ48132474,image-pre-inspection-serial-number-a2dc2e0b-7a...,data/Laundry/SJ48132474/image-pre-inspection-s...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M,14,15
3,SJ48132474,image-pre-inspection-serial-number-d843dfb6-fb...,data/Laundry/SJ48132474/image-pre-inspection-s...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M,14,15
4,SJ48144276,image-pre-inspection-proof-of-purchase-8e82720...,data/Laundry/SJ48144276/image-pre-inspection-p...,HOME LAUNDRY GAS,GTD42GASJ2WW,GTD42GASJ2WW,12,12


In [41]:
df1['sn_len'].unique()

array([15, 12,  9, 11, 10,  8,  7, 14])

In [42]:
df2 = df1.loc[:,('PRODUCT_LINE', 'MODEL_NUMBER', 'SERIAL_NUMBER', 'mod_num_len', 'sn_len')]
df2.head()

,PRODUCT_LINE,MODEL_NUMBER,SERIAL_NUMBER,mod_num_len,sn_len
0,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M,14,15
1,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M,14,15
2,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M,14,15
3,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M,14,15
4,HOME LAUNDRY GAS,GTD42GASJ2WW,GTD42GASJ2WW,12,12


In [43]:
df2.groupby('PRODUCT_LINE')['mod_num_len','sn_len'].agg(['unique'])

<ipython-input-43-ae9744cbd896>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2.groupby('PRODUCT_LINE')['mod_num_len','sn_len'].agg(['unique'])


,mod_num_len,sn_len
,unique,unique
PRODUCT_LINE,,
DISHWASHER,"[11, 14, 13, 6, 12, 18, 9]","[9, 15, 7, 14, 8, 10, 12]"
HOME LAUNDRY ELEC,"[10, 12, 9, 11, 14, 13, 8]","[9, 15, 10, 12, 14]"
HOME LAUNDRY GAS,"[14, 12, 8, 11, 10, 16, 9]","[15, 12, 9, 11, 10, 8]"
HOME REFRIGERATION,"[10, 11, 12, 13, 14]","[10, 15, 9, 12]"


In [44]:
df2.query('PRODUCT_LINE == "DISHWASHER" & sn_len == 7')

,PRODUCT_LINE,MODEL_NUMBER,SERIAL_NUMBER,mod_num_len,sn_len
82,DISHWASHER,SHPM65W55N/11,SN6AP5B,13,7
83,DISHWASHER,SHPM65W55N/11,SN6AP5B,13,7


In [45]:
print(df.query('SERIAL_NUMBER == "SN6AP5B"')['example'].values)

['data/Laundry/SJ48479471/image-pre-inspection-photo-48104256-eb4a-4829-8e21-a0a49276a73f_dae659e4-a0f6-4816-a8e2-772e6d44306e.png'
 'data/Laundry/SJ48479471/image-pre-inspection-photo-f2582027-f58d-4248-888b-52580c2cb89a_dae659e4-a0f6-4816-a8e2-772e6d44306e.png']


In [46]:
fn_df.query('FILE_NAME == "image-pre-inspection-photo-48104256-eb4a-4829-8e21-a0a49276a73f_dae659e4-a0f6-4816-a8e2-772e6d44306e.png"')

,SN,FILE_NAME,example
283,284,image-pre-inspection-photo-48104256-eb4a-4829-...,data/Laundry/SJ48479471/image-pre-inspection-p...


In [47]:
fn_df.head()

,SN,FILE_NAME,example
0,1,image-pre-inspection-serial-number-e251c60d-82...,data/SJ48263644/image-pre-inspection-serial-nu...
1,2,image-pre-inspection-serial-number-06b7b5d3-63...,data/SJ49160346/image-pre-inspection-serial-nu...
2,3,image-pre-inspection-serial-number-cdabb4cc-13...,data/SJ48161309/image-pre-inspection-serial-nu...
3,4,image-pre-inspection-proof-of-purchase-03327aa...,data/SJ48161309/image-pre-inspection-proof-of-...
4,5,image-pre-inspection-serial-number-f4accc50-1f...,data/SJ48326900/image-pre-inspection-serial-nu...


In [48]:
df0 = df1.drop_duplicates(subset='MODEL_NUMBER', keep='last')
df0.head()

,SERVICE_JOB_ID,FILE_NAME,example,PRODUCT_LINE,MODEL_NUMBER,SERIAL_NUMBER,mod_num_len,sn_len
3,SJ48132474,image-pre-inspection-serial-number-d843dfb6-fb...,data/Laundry/SJ48132474/image-pre-inspection-s...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M,14,15
10,SJ48165170,image-pre-inspection-photo-3cdfaeaf-fd4b-485e-...,data/Laundry/SJ48165170/image-pre-inspection-p...,HOME LAUNDRY GAS,00000000,00000000000,8,11
16,SJ48173268,image-pre-inspection-serial-number-b6cfaae5-3d...,data/Laundry/SJ48173268/image-pre-inspection-s...,HOME LAUNDRY GAS,EFMG527UIW1,4D12607202,11,10
30,SJ48293855,image-pre-inspection-photo-c1b73a06-f945-49f5-...,data/Laundry/SJ48293855/image-pre-inspection-p...,HOME LAUNDRY ELEC,WFW85HEFC1,C80550258,10,9
32,SJ48294169,image-pre-inspection-serial-number-db49d094-76...,data/Laundry/SJ48294169/image-pre-inspection-s...,HOME LAUNDRY GAS,GTX42GASJ2WW,HS808910C,12,9


In [49]:
unique_images_df = fn_df[fn_df['FILE_NAME'].isin(df0["FILE_NAME"])]
unique_images_df.head()

,SN,FILE_NAME,example
124,125,image-pre-inspection-serial-number-6c95b3ce-22...,data/Laundry/SJ49039066/image-pre-inspection-s...
158,159,image-pre-inspection-serial-number-f23170e9-b6...,data/Laundry/SJ49026246/image-pre-inspection-s...
165,166,image-pre-inspection-serial-number-a97cc190-55...,data/Laundry/SJ48852406/image-pre-inspection-s...
181,182,image-pre-inspection-serial-number-0c3a1911-b8...,data/Laundry/SJ49074656/image-pre-inspection-s...
185,186,image-pre-inspection-serial-number-a6c2bd35-39...,data/Laundry/SJ49050408/image-pre-inspection-s...


In [19]:
# loop over photos manually to see if keywords are present
# for i in unique_images_df['SN'].tolist():
#     img = cv2.imread(f'./new_data/{i}.png')
#     cv2.imshow('Original', img)
#     cv2.waitKey(0)

# cv2.destroyAllWindows()
